In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('train.csv')
age_gender_df = pd.read_csv('age_gender_info.csv')
test_df = pd.read_csv('test.csv')

In [3]:
def data_preprocessing(train_df, test_df):
    
    # 오류 2번, 3번만 제거 : 1번은 감안하여 분석 하세요!
    error_data = error_data = ['C2085', 'C1397', 'C2431', 'C1649', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    
    for error in error_data :
        train_df = train_df[train_df['단지코드'] != error]
    
    train_df.loc[train_df['임대보증금'] == '-','임대보증금'] = np.nan
    train_df.loc[train_df['임대료'] == '-','임대료'] = np.nan

    train_df['임대보증금'] = train_df['임대보증금'].astype(float)
    train_df['임대료'] = train_df['임대료'].astype(float)
    
    cols = ['임대보증금', '임대료','도보 10분거리 내 지하철역 수(환승노선 수 반영)','도보 10분거리 내 버스정류장 수']

    train_df[cols] = train_df[cols].fillna(0)
    test_df[cols] = test_df[cols].fillna(0)
    
    test_df[ (test_df['단지코드']=='C2411') & (test_df['자격유형'].isnull())] = 'A'
    test_df[ (test_df['단지코드']=='C2253') & (test_df['자격유형'].isnull())] = 'C'
    
    train_df = train_df.drop_duplicates()
    test_df = test_df.drop_duplicates()
    
    
    return train_df, test_df

In [14]:
train, test = data_preprocessing(train_df, test_df)

In [15]:
train.set_index('단지코드', drop=True,inplace=True)

In [16]:
trainDF = pd.get_dummies(train)

In [17]:
trainDF

,총세대수,전용면적,전용면적별세대수,공가수,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,...,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O
단지코드,,,,,,,,,,,,,,,,,,,,,
C2515,545,33.48,276,17.0,9216000.0,82940.0,0.0,3.0,624.0,205.0,...,0,0,0,0,0,0,0,0,0,0
C2515,545,39.60,60,17.0,12672000.0,107130.0,0.0,3.0,624.0,205.0,...,0,0,0,0,0,0,0,0,0,0
C2515,545,39.60,20,17.0,12672000.0,107130.0,0.0,3.0,624.0,205.0,...,0,0,0,0,0,0,0,0,0,0
C2515,545,46.90,38,17.0,18433000.0,149760.0,0.0,3.0,624.0,205.0,...,0,0,0,0,0,0,0,0,0,0
C2515,545,46.90,19,17.0,18433000.0,149760.0,0.0,3.0,624.0,205.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2532,239,49.20,19,7.0,11346000.0,116090.0,0.0,1.0,166.0,146.0,...,0,0,0,0,0,0,0,0,0,0
C2532,239,51.08,34,7.0,14005000.0,142310.0,0.0,1.0,166.0,146.0,...,0,0,0,0,0,0,0,0,0,0
C2532,239,51.73,34,7.0,14005000.0,142310.0,0.0,1.0,166.0,146.0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

y_target = trainDF['등록차량수']
X_data = trainDF.drop(['등록차량수'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=0)

In [19]:
rf = RandomForestRegressor(random_state=0, n_estimators=1000)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
neg_mae_scores = mean_absolute_error(y_test,y_pred)

In [20]:
neg_mae_scores

27.048766623207303